In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


In [3]:
diabetes.shape

(778, 10)

Exercise 8.1: \
Начнём с поиска дубликатов в данных. \
Найдите все повторяющиеся строки в данных и удалите их. \
Для поиска используйте все признаки в данных. \
Сколько записей осталось в данных?

In [4]:
#set up the columns to be reviewed
dupl_columns = list(diabetes.columns)

#use the series to find the duplicates
mask = diabetes.duplicated(subset=dupl_columns)
sber_duplicates = diabetes[mask]
print(f'No. of Duplicates: {sber_duplicates.shape[0]}')

No. of Duplicates: 10


In [5]:
#create a new table free of duplicates
diabetes_dedupped = diabetes.drop_duplicates(subset=dupl_columns)
print(f'New No. of Records: {diabetes_dedupped.shape[0]}')

New No. of Records: 768


Exercise 8.2: \
Далее найдите все неинформативные признаки в данных и избавьтесь от них. \
В качестве порога информативности возьмите 0.95: удалите все признаки, для которых 95% значений повторяются или 95% записей уникальны. \
В ответ запишите имена признаков, которые вы нашли (без кавычек).

In [6]:
#list of non-informative features
low_information_cols = [] 

#loop through all the columns in a non-duplicated table
for col in diabetes_dedupped.columns:
    #highest relative frequency in a feature
    top_freq = diabetes_dedupped[col].value_counts(normalize=True).max()
    #proportion of unique values from feature size
    nunique_ratio = diabetes_dedupped[col].nunique() / diabetes_dedupped[col].count()
    #compare the highest frequency with the threshold
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% identical values')
    #compare the proportion of unique values with a threshold
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% unique values')

Gender: 100.0% identical values


In [7]:
#delete non-informative features
diabetes_dedupped = diabetes_dedupped.drop(low_information_cols, axis=1)
print(f'Resulting No. of Features: {diabetes_dedupped.shape[1]}')

Resulting No. of Features: 9


Exercise 8.3: \
Попробуйте найти пропуски в данных с помощью метода isnull(). \
Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. \
Какая доля пропусков содержится в столбце Insulin? \
Ответ округлите до сотых.

In [8]:
#apply isnull() method on all columns
cols_null_percent = diabetes_dedupped.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null)

Series([], dtype: float64)

In [9]:
lst_cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
#производим замену через маску
diabetes_dedupped[diabetes_dedupped[lst_cols] == 0] = np.nan

In [10]:
#check that the function worked correctly
#apply isnull() method on all columns
cols_null_percent = diabetes_dedupped.isnull().mean()
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null)

Insulin          0.486979
SkinThickness    0.295573
BloodPressure    0.045573
BMI              0.014323
Glucose          0.006510
dtype: float64

Exercise 8.4: \
Удалите из данных признаки, где число пропусков составляет более 30%. \
Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?

In [11]:
#set the min threshhold: calculate 70% from total number of rows
thresh = diabetes_dedupped.shape[0]*0.7
#delete columns with more than 30% (100-70) null values / blanks
diabetes_dedupped = diabetes_dedupped.dropna(thresh=thresh, axis=1)

In [12]:
print(f'Resulting No. of Features: {diabetes_dedupped.shape[1]}')

Resulting No. of Features: 8


Exercise 8.5: \
Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. \
Чему равно результирующее число записей в таблице?

In [13]:
#отбрасываем строки с числом пропусков более 2 в строке
m = diabetes_dedupped.shape[1] #число признаков после удаления столбцов
diabetes_dedupped = diabetes_dedupped.dropna(thresh=m-2, axis=0)
diabetes_dedupped.shape

(761, 8)

In [14]:
diabetes_dedupped.isnull().mean()

Pregnancies                 0.000000
Glucose                     0.006570
BloodPressure               0.036794
SkinThickness               0.289093
BMI                         0.005256
DiabetesPedigreeFunction    0.000000
Age                         0.000000
Outcome                     0.000000
dtype: float64

Exercise 8.6: \
В оставшихся записях замените пропуски на медиану. \
Чему равно среднее значение в столбце SkinThickness? \
Ответ округлите до десятых. 

In [15]:
#create dict 'column name': number (feature) that needs to replace blanks for
values = {
    'Glucose': diabetes_dedupped['Glucose'].median(),
    'BloodPressure': diabetes_dedupped['BloodPressure'].median(),
    'SkinThickness': diabetes_dedupped['SkinThickness'].median(),
    'BMI': diabetes_dedupped['BMI'].median()
}
#заполняем оставшиеся записи константами в соответствии со словарем values
diabetes_dedupped = diabetes_dedupped.fillna(values)
#выводим результирующую долю пропусков
display(diabetes_dedupped.isnull().mean())

Pregnancies                 0.0
Glucose                     0.0
BloodPressure               0.0
SkinThickness               0.0
BMI                         0.0
DiabetesPedigreeFunction    0.0
Age                         0.0
Outcome                     0.0
dtype: float64

In [16]:
diabetes_dedupped['SkinThickness'].mean().round(1)

29.1

Exercise 8.7: \
Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?

In [17]:
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [18]:
outliers, cleaned = outliers_iqr(diabetes_dedupped, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


Exercise 8.8

In [19]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [20]:
outliers, cleaned = outliers_z_score(diabetes_dedupped, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 4
Результирующее число записей: 757


Exercise 8.9: \
На приведённой гистограмме показано распределение признака DiabetesPedigreeFunction. \
Такой вид распределения очень похож на логнормальный, и он заставляет задуматься о логарифмировании признака. \
Найдите сначала число выбросов в признаке DiabetesPedigreeFunction с помощью классического метода межквартильного размаха.

In [21]:
outliers, cleaned = outliers_iqr(diabetes_dedupped, 'DiabetesPedigreeFunction')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 29
Результирующее число записей: 732


Затем найдите число выбросов в этом же признаке в логарифмическом масштабе (при логарифмировании единицу прибавлять не нужно!). \
Какова разница между двумя этими числами (вычтите из первого второе)?

In [22]:
#modified function
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr_mod(diabetes_dedupped, 'DiabetesPedigreeFunction', log_scale=True)
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')

Число выбросов по методу Тьюки: 0
